<a href="https://colab.research.google.com/github/wael-slimi/Speciality_Recommedation/blob/main/Speciality_Recommedation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'doctor-specialist-recommendation-system:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3417941%2F5969573%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240425%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240425T020157Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D10edc02ee98f39bb687a0e1c762b67d5ebd87ce4f6bf704f1d33de5ecfc7690e6924588b881146388e164fce78529ea2a769f61da3f56cc2fca13be64e609618065f48bec268eab39cf67149ca7c4df4da0e4167b5ffdf30fb4d0a6a4184425ebaa1ae0dfca892445cf7ddc76df4a88fea34fb6a212fff9cab643dacf9ca25ffe276a5d54d4bcd88ed3e7ce7a6edf7b595a9b436a918c7ddb5a98c8ce904e203f5e111aebdfcacea61f0185a55b76224706719ab84b04ab2fbfb8ffea4aa1d17fa5612740ca811c1aecfad98e5194bc987759798b72ed00378f1f1b544fd49396633a496043573bac6a1d135fa2931bd680cf8895ea801fe5c1a472807fde133'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


### Predicting Disease based on the Symptoms and identifying specialist based on predicted disease

**The dataset contains information about symptoms, diseases, recommended doctors for each disease, and descriptions of the diseases. Using a classification model, diseases have been predicted based on the given symptoms. The predictions provide insights into the predicted disease, the likelihood or chance of that disease based on the classification model, the recommended doctor to visit for that disease, and a description of the predicted disease.**

**The summary output includes:**

**Predicted Disease:** *-The disease that has been predicted based on the symptoms provided.-*

**Chance of the Disease:** *-The likelihood or probability of having that disease, as determined by the classification model.-*

**Recommended Doctor:** *-The doctor or specialist recommended to visit for further evaluation or treatment of the predicted disease.-*

**Description of the Disease:** *-A brief overview or description of the predicted disease, providing additional information about its symptoms, causes, and potential treatments.-*

**This summary aims to provide a concise and informative overview of the predicted disease, its associated likelihood, the recommended doctor, and a brief description to aid in understanding and decision-making regarding the predicted disease.**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from collections import Counter

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
dis_sym_data = pd.read_csv("/kaggle/input/doctor-specialist-recommendation-system/Original_Dataset.csv")

In [ ]:
dis_sym_data.head()

In [ ]:
dis_sym_data.shape

**Finding unique values across all the symptoms column**

In [ ]:
columns_to_check = []
for col in dis_sym_data.columns:
    if col != 'Disease':
        columns_to_check.append(col)

In [ ]:
symptoms = dis_sym_data.iloc[:, 1:].values.flatten()
symptoms = list(set(symptoms))

**Convert Symptoms to Binary Columns**

In [ ]:
for symptom in symptoms:
    dis_sym_data[symptom] = dis_sym_data.iloc[:, 1:].apply(lambda row: int(symptom in row.values), axis=1)

dis_sym_data_v1 = dis_sym_data.drop(columns=columns_to_check)

In [ ]:
dis_sym_data_v1 = dis_sym_data_v1.loc[:, dis_sym_data_v1.columns.notna()]

In [ ]:
dis_sym_data_v1.shape

In [ ]:
dis_sym_data_v1.columns = dis_sym_data_v1.columns.str.strip()

In [ ]:
dis_sym_data_v1.columns

**Encoding Disease**

In [ ]:
var_mod = ['Disease']
le = LabelEncoder()
for i in var_mod:
    dis_sym_data_v1[i] = le.fit_transform(dis_sym_data_v1[i])

In [ ]:
X = dis_sym_data_v1.drop(columns="Disease")
y = dis_sym_data_v1['Disease']

**Model Generation**

In [ ]:
def class_algo(model,independent,dependent):
    model.fit(independent,dependent)
    pred = model.predict(independent)
    accuracy = metrics.accuracy_score(pred,dependent)
    print(model_name,'Accuracy : %s' % '{0:.3%}'.format(accuracy))

In [ ]:
algorithms = {'Logistic Regression':
              {"model": LogisticRegression()},

              'Decision Tree':
              {"model": tree.DecisionTreeClassifier()},

              'Random Forest':
              {"model": RandomForestClassifier()},

              'SVM':
              {"model": svm.SVC(probability=True)},

              'NaiveBayes' :
              {"model": GaussianNB()},

              'K-Nearest Neighbors' :
              {"model": KNeighborsClassifier()},
             }

In [ ]:
for model_name, values in algorithms.items():
    class_algo(values["model"],X,y)

**Map Description and Specialist for the Disease Predicted**

In [ ]:
doc_data = pd.read_csv("/kaggle/input/doctor-specialist-recommendation-system/Doctor_Versus_Disease.csv",encoding='latin1', names=['Disease','Specialist'])

In [ ]:
doc_data.tail(5)

 -*Doctor name is wrong for Disease - Tuberculosis. Replace to Pulmonologist*-

In [ ]:
doc_data['Specialist'] = np.where((doc_data['Disease'] == 'Tuberculosis'),'Pulmonologist', doc_data['Specialist'])

In [ ]:
doc_data.tail(5)

In [ ]:
des_data = pd.read_csv("/kaggle/input/doctor-specialist-recommendation-system/Disease_Description.csv")

In [ ]:
des_data.head()

**Test with Unknown Data**

In [ ]:
test_col = []
for col in dis_sym_data_v1.columns:
    if col != 'Disease':
        test_col.append(col)


test_data = {}
symptoms = []
predicted = []
def test_input():
    symptoms.clear()
    predicted.clear()
    num_inputs = int(input("Enter the number of symptoms you have: "))
    for i in range(num_inputs):
        user_input = input("Enter Symptoms #{}: ".format(i+1))
        symptoms.append(user_input)
    print("Symptoms you have:", symptoms)
    for column in test_col:
        test_data[column] = 1 if column in symptoms else 0
        test_df = pd.DataFrame(test_data, index=[0])
    print("Predicting Disease based on 6 ML algorithms...")
    for model_name, values in algorithms.items():
        predict_disease = values["model"].predict(test_df)
        predict_disease = le.inverse_transform(predict_disease)
        predicted.extend(predict_disease)
    disease_counts = Counter(predicted)
    percentage_per_disease = {disease: (count / 6) * 100 for disease, count in disease_counts.items()}
    result_df = pd.DataFrame({"Disease": list(percentage_per_disease.keys()),
                               "Chances": list(percentage_per_disease.values())})
    result_df = result_df.merge(doc_data, on='Disease', how='left')
    result_df = result_df.merge(des_data, on='Disease', how='left')
    return result_df

In [ ]:
test_input()